In [ ]:
pip install transformers datasets evaluate seqeval

# Task

Token classification assigns a label to individual tokens in a sentence. One of the most common token classification tasks is Named Entity Recognition (NER). NER attempts to find a label for each entity in a sentence, such as a person, location, or organization.

This notebook shows how to:
1. Finetune DistilBERT on the WNUT 17 dataset to detect new entities.
2. Use the finetuned model for inference.

# Libraries

In [ ]:
from datasets import load_dataset

# Data

In [ ]:
# Load the WNUT 17 dataset from the 🤗 Datasets library
wnut = load_dataset("wnut_17")